#### CSCE 670 :: Information Storage & Retrieval :: Texas A&M University :: Spring 2025


# Homework 1:  OASIS Search Engine, The Beginning

### 100 points [4% of your final grade]

### Due: February 5 (Wednesday) by 11:59pm

*Goals of this homework:* In this homework you will get first hand experience building a text-based mini search engine + explore some LLM capabilities.

*Submission instructions (Canvas):* To submit your homework, rename this notebook as `UIN_hw1.ipynb`. For example, my homework submission would be something like `555001234_hw1.ipynb`. Submit this notebook via Canvas (look for the homework 1 assignment there). Your notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so). So please run all the cells for us, and then submit.

*Late submission policy:* For this homework, you may use as many late days as you like (up to the 5 total allotted to you).

*Collaboration policy:* You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Canvas, search StackOverflow, even use ChatGPT. But if you do get help in this way, you must inform us by **filling out the Collaboration Declarations at the bottom of this notebook**. See the course syllabus for details.

*Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.*

The basic rule is that no student should explicitly share a solution with another student (and thereby circumvent the basic learning process), but it is okay to share general approaches, directions, and so on. If you feel like you have an issue that needs clarification, feel free to contact either me or the TA.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Dataset: Enron Email Dataset

We are providing you with a small collection of emails from the Enron Email Dataset. The Enron Email Dataset was collected and prepared by the CALO Project (A Cognitive Assistant that Learns and Organizes). It contains data from about 150 users, mostly senior management of Enron. The full corpus contains a total of about 0.5M messages (https://www.cs.cmu.edu/~enron/). For this homework, we will use a small subset of the data. The subset contains 814 emails extracted from the `_sent_mail` of Arnold-J. We have zipped the 814 files (each file contains the information of an email). The zipped file is available on Canvas as `enron_814.zip`. The subset we provide is about 1.1MB. You should treat each email as a unique document to be indexed by your system. You can download the data from Canvas to your local filesystem. We're going to use these emails as the basis of OASIS, our Open Access Searchable Information System!


Below is an example of one email.

```text
Message-ID: <33025919.1075857594206.JavaMail.evans@thyme>
Date: Wed, 13 Dec 2000 13:09:00 -0800 (PST)
From: john.arnold@enron.com
To: slafontaine@globalp.com
Subject: re:spreads
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: John Arnold
X-To: slafontaine@globalp.com @ ENRON
X-cc:
X-bcc:
X-Folder: \John_Arnold_Dec2000\Notes Folders\'sent mail
X-Origin: Arnold-J
X-FileName: Jarnold.nsf

saw a lot of the bulls sell summer against length in front to mitigate
margins/absolute position limits/var.  as these guys are taking off the
front, they are also buying back summer.  el paso large buyer of next winter
today taking off spreads.  certainly a reason why the spreads were so strong
on the way up and such a piece now.   really the only one left with any risk
premium built in is h/j now.   it was trading equivalent of 180 on access,
down 40+ from this morning.  certainly if we are entering a period of bearish
to neutral trade, h/j will get whacked.  certainly understand the arguments
for h/j.  if h settles $20, that spread is probably worth $10.  H 20 call was
trading for 55 on monday.  today it was 10/17.  the market's view of
probability of h going crazy has certainly changed in past 48 hours and that
has to be reflected in h/j.




slafontaine@globalp.com on 12/13/2000 04:15:51 PM
To: slafontaine@globalp.com
cc: John.Arnold@enron.com
Subject: re:spreads



mkt getting a little more bearish the back of winter i think-if we get another
cold blast jan/feb mite move out. with oil moving down and march closer flat
px
wide to jan im not so bearish these sprds now-less bullish march april as
well.
```

# Part 1: Read and Parse the Email Data (20 points)

Recall how we handled file input in Homework 0? Well, here, our goal is to read the emails so that we can begin to tokenize them later. For this step, you should read the dataset and print the emails. Note that our dataset contains multiple files. You will need to write Python code to read from these files, and then build a list to store the documents. Each item in the list should be a dictionary containing the `Document-ID` as the key, and email content as the value. You can discard all the supplementary information of the email, e.g., `Date`, `From`, `To`, `Subject`, etc.

A document should look like:

```text
{'Document-ID': '33025919.1075857594206',
'content': 'saw a lot of the bulls sell summer against length in front to mitigate
margins/absolute position limits/var.  as these guys are taking off the
front, they are also buying back summer.  el paso large buyer of next winter
today taking off spreads.  certainly a reason why the spreads were so strong
on the way up and such a piece now.   really the only one left with any risk
premium built in is h/j now.   it was trading equivalent of 180 on access,
down 40+ from this morning.  certainly if we are entering a period of bearish
to neutral trade, h/j will get whacked.  certainly understand the arguments
for h/j.  if h settles $20, that spread is probably worth $10.  H 20 call was
trading for 55 on monday.  today it was 10/17.  the market's view of
probability of h going crazy has certainly changed in past 48 hours and that
has to be reflected in h/j.




slafontaine@globalp.com on 12/13/2000 04:15:51 PM
To: slafontaine@globalp.com
cc: John.Arnold@enron.com
Subject: re:spreads



mkt getting a little more bearish the back of winter i think-if we get another
cold blast jan/feb mite move out. with oil moving down and march closer flat
px
wide to jan im not so bearish these sprds now-less bullish march april as
well.'
}
```

For this homework, you should treat the email content as a document and the Message-ID as the document ID.

## Print the first two documents (5 points)

Your output should look like this:

DocumentID Document

33025919.1075857594206 saw a lot of the bulls sell summer ......

...


In [2]:
# your code here
# please print out the first 2 docs
import os
import re

def read_emails_from_directory(directory):
    emails = []

    for i, filename in enumerate(os.listdir(directory)):
        file_path = os.path.join(directory, filename)

        if os.path.isfile(file_path):
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
                content = file.read()

                # Extract Message-ID using regex
                message_id_match = re.search(r"Message-ID:\s*<([\d\.]+)", content)  # Extract only the numeric part
                message_id = message_id_match.group(1).rstrip('.')  # Extract Message-ID

                # Remove headers using regex
                # After observing the email structure I can see a pattern where the body and header of any email is seperated by 2 newlines i.e \n\n
                email_body = re.split(r'\n\n', content, maxsplit=1)
                if len(email_body) > 1:
                    email_text = email_body[1]  # Extract content after headers
                else:
                    email_text = email_body[0]

                emails.append({"Document-ID": message_id, "content": email_text.strip()})

    return emails

# Example usage
dataset_directory = "/content/drive/MyDrive/ISR/HW1/enron_814"
email_data = read_emails_from_directory(dataset_directory)

# Print contents of Documents '1' and '2'
for email in email_data:
    if email["Document-ID"] in ["33025919.1075857594206", "9214363.1075857594228"]:
      print(email)

{'Document-ID': '33025919.1075857594206', 'content': "saw a lot of the bulls sell summer against length in front to mitigate \nmargins/absolute position limits/var.  as these guys are taking off the \nfront, they are also buying back summer.  el paso large buyer of next winter \ntoday taking off spreads.  certainly a reason why the spreads were so strong \non the way up and such a piece now.   really the only one left with any risk \npremium built in is h/j now.   it was trading equivalent of 180 on access, \ndown 40+ from this morning.  certainly if we are entering a period of bearish \nto neutral trade, h/j will get whacked.  certainly understand the arguments \nfor h/j.  if h settles $20, that spread is probably worth $10.  H 20 call was \ntrading for 55 on monday.  today it was 10/17.  the market's view of \nprobability of h going crazy has certainly changed in past 48 hours and that \nhas to be reflected in h/j.\n\n\n\n\nslafontaine@globalp.com on 12/13/2000 04:15:51 PM\nTo: slafo

Now that you can read the documents, let's move on to tokenization. We are going to simplify things for you:
1. You should **lowercase** all words.
2. Replace line breaks (e.g., \n, \n\n), punctuations, dashes and splash (e.g., -, /) and special characters (\u2019, \u2005, etc.) with empty space (" ").
3. Tokenize the documents by splitting on whitespace.
4. Then only keep words that have a-zA-Z in them.

In [3]:
# your code here
import re

def preprocess_email_content(email_text):
    # Convert to lowercase
    email_text = email_text.lower()

    # Remove special characters, punctuation, dashes, slashes, etc.
    email_text = re.sub(r'[\n\r\t\-/\\\u2019\u2005\u00A0\u2013\u2014\u2026.,!+=*^%$?;:"\'(){}\[\]<>#@&|~]', ' ', email_text)

    # Tokenize by splitting on whitespace
    tokens = email_text.split()

    # Keep only words containing a-z or A-Z
    tokens = [word for word in tokens if re.search(r'[a-zA-Z]', word)]

    return tokens

# Apply preprocessing to extracted emails
for email in email_data:
    email["content"] = preprocess_email_content(email["content"])

## Print the first two documents after tokenizing (5 points)

Once you have your parser working, you should print the first two documents (documentID and tokens).

Your output should look like this:

* DocumentID Tokens



In [4]:
# your code and output here

# Print processed emails for Document-ID '1' and '2'
for email in email_data:
    if email["Document-ID"] in ["33025919.1075857594206", "9214363.1075857594228"]:
        print(email)

{'Document-ID': '33025919.1075857594206', 'content': ['saw', 'a', 'lot', 'of', 'the', 'bulls', 'sell', 'summer', 'against', 'length', 'in', 'front', 'to', 'mitigate', 'margins', 'absolute', 'position', 'limits', 'var', 'as', 'these', 'guys', 'are', 'taking', 'off', 'the', 'front', 'they', 'are', 'also', 'buying', 'back', 'summer', 'el', 'paso', 'large', 'buyer', 'of', 'next', 'winter', 'today', 'taking', 'off', 'spreads', 'certainly', 'a', 'reason', 'why', 'the', 'spreads', 'were', 'so', 'strong', 'on', 'the', 'way', 'up', 'and', 'such', 'a', 'piece', 'now', 'really', 'the', 'only', 'one', 'left', 'with', 'any', 'risk', 'premium', 'built', 'in', 'is', 'h', 'j', 'now', 'it', 'was', 'trading', 'equivalent', 'of', 'on', 'access', 'down', 'from', 'this', 'morning', 'certainly', 'if', 'we', 'are', 'entering', 'a', 'period', 'of', 'bearish', 'to', 'neutral', 'trade', 'h', 'j', 'will', 'get', 'whacked', 'certainly', 'understand', 'the', 'arguments', 'for', 'h', 'j', 'if', 'h', 'settles', 'tha

## Dictionary Size (5 points)

Next you should report the size of your dictionary, that is, how many unique tokens among all the documents.




In [5]:
# your code and output here

# Create a set to store unique tokens
unique_tokens = set()
all_tokens = []

# Processed emails and stored in email["content"] from previous step. After that add tokens to the set
for email in email_data:
    tokens = email["content"]
    all_tokens.append(tokens)
    unique_tokens.update(tokens)

# Report dictionary size
dictionary_size = len(unique_tokens)
print(f"Dictionary Size: {dictionary_size}")


Dictionary Size: 9737


## Top-20 Words (5 points)

Finally, you should print a list of the top-20 most popular words by counting among all documents.


Your output should look like this:

* Rank. Token, Count

1. awesome, 20
2. cool, 15
3. ...

In [6]:
# your code and output here

# Initialize an empty dictionary to store word frequencies
word_counts = {}

# Process each email and update word counts manually
for token in all_tokens:
    for word in token:
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

# Sort the dictionary by frequency in descending order
sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

# Get the top-20 most frequent words
top_20_words = sorted_word_counts[:20]

# Print the results in the required format
print("\nTop-20 Most Frequent Words:\n")
for rank, (word, count) in enumerate(top_20_words, start=1):
    print(f"{rank}. {word}, {count}")



Top-20 Most Frequent Words:

1. the, 4111
2. to, 3768
3. ect, 2600
4. enron, 2241
5. a, 1879
6. and, 1795
7. you, 1749
8. of, 1716
9. i, 1694
10. on, 1590
11. john, 1505
12. in, 1465
13. hou, 1345
14. com, 1317
15. is, 1202
16. for, 1165
17. arnold, 991
18. subject, 936
19. s, 867
20. it, 865


# Part 2: Boolean Retrieval (30 points)

In this part you will build an inverted index to support Boolean retrieval. You should use the tokenization strategy from above.

We require your index to support AND, OR, NOT queries.

Search for the queries below using your index and print out matching documents (for each query, print out 5 matching documents):
* buyer
* margins AND limits
* winter OR summer
* buyers AND risk AND NOT crazy
* never OR know

Recall, that you should apply the exact same pre-processing strategies to the query as we do to the documents.

The output should like this:
* DocumentID Document

To make our life easier, please output the DocumentIDs in lexicographic order.

In [7]:
# build the index here
# add cells as needed to organize your code

# Step 1: Build the inverted index
from collections import defaultdict

# Initialize inverted index as a dictionary where each token maps to a set of document IDs
inverted_index = defaultdict(set)

# Populate the inverted index
for email in email_data:
    doc_id = email["Document-ID"]
    tokens = set(email["content"])  # Use a set to avoid duplicate entries in a document
    for token in tokens:
        inverted_index[token].add(doc_id)

# Step 2: Define Boolean Query Functions
def boolean_and(set1, set2):
    return set1 & set2  # Intersection (AND operation)

def boolean_or(set1, set2):
    return set1 | set2  # Union (OR operation)

def boolean_not(set1, all_docs):
    return all_docs - set1  # Complement (NOT operation)


In [8]:
# Step 3: Process Queries
def process_query(query, inverted_index, all_docs):
    # Process a Boolean query with proper precedence handling.
    query_tokens = preprocess_email_content(query)  # Tokenize and lowercase query
    if not query_tokens:
        return set()

    stack = []
    operators = []

    def apply_operator():
        # Apply the last operator in the stack on the last two sets in the stack.
        if len(stack) < 2 and operators[-1] != "not":
            return
        op = operators.pop()
        if op == "and":
            right = stack.pop()
            left = stack.pop()
            stack.append(boolean_and(left, right))
        elif op == "or":
            right = stack.pop()
            left = stack.pop()
            stack.append(boolean_or(left, right))
        elif op == "not":
            right = stack.pop()
            stack.append(boolean_not(right, all_docs))

    # Process tokens with operator precedence
    # NOT has the highest precedence
    for token in query_tokens:
        if token in {"and", "or", "not"}:
            while operators and operators[-1] == "not":
                apply_operator()
            operators.append(token)
        else:
            stack.append(inverted_index.get(token, set()))

    while operators:
        apply_operator()

    return stack[0] if stack else set()

all_docs = {email["Document-ID"] for email in email_data}  # Set of all document IDs

## Running the five queries (4 points each, 20 points in total)

In [9]:
# search for the input using your index and print out ids of matching documents.

Now show the results for the query: `buyer`

In [10]:
# your code here
query = "buyer"
matching_docs = sorted(process_query(query, inverted_index, all_docs))[:5]
print(f"\nQuery: {query}\nMatching Documents:")
for doc_id in matching_docs:
    email_content = next(email["content"] for email in email_data if email["Document-ID"] == doc_id)
    print(f"{doc_id}: {' '.join(email_content)}")


Query: buyer
Matching Documents:
13960264.1075857658698: very useful thx keep me posted caroline abramo enron am to john arnold hou ect ect mike maggi corp enron enron jennifer fraser hou ect ect cc per sekse ny ect ect subject fund views hi all the funds are trying to figure out what the play is for next year major divergence of opinions most everyone we talk to takes a macro view dwight anderson from tudor thinks anything above is a sale from the perspective of shut in industrial demand he believes that between no industrial basic industry type can operate he tracks all the plant closures similar to what elena does in mike robert s group but it seems on a more comprehensive level jen it would be good for fundamentals to track this number do some scenario analysis on it under various economic conditions like recession i will try to find out what his total number for turned back gas is just ammonia is a little more than bcf which does not seem all that meaningful but the total may bri

*Now* show the results for the query: `margins AND limits`

---




In [11]:
# your code here
query = "margins AND limits"
matching_docs = sorted(process_query(query, inverted_index, all_docs))[:5]
print(f"\nQuery: {query}\nMatching Documents:")
for doc_id in matching_docs:
    email_content = next(email["content"] for email in email_data if email["Document-ID"] == doc_id)
    print(f"{doc_id}: {' '.join(email_content)}")  # Print only first 30 words for preview


Query: margins AND limits
Matching Documents:
33025919.1075857594206: saw a lot of the bulls sell summer against length in front to mitigate margins absolute position limits var as these guys are taking off the front they are also buying back summer el paso large buyer of next winter today taking off spreads certainly a reason why the spreads were so strong on the way up and such a piece now really the only one left with any risk premium built in is h j now it was trading equivalent of on access down from this morning certainly if we are entering a period of bearish to neutral trade h j will get whacked certainly understand the arguments for h j if h settles that spread is probably worth h call was trading for on monday today it was the market s view of probability of h going crazy has certainly changed in past hours and that has to be reflected in h j slafontaine globalp com on pm to slafontaine globalp com cc john arnold enron com subject re spreads mkt getting a little more bearish

Now show the results for the query: `winter OR summer`

In [12]:
# your code here
query = "winter OR summer"
matching_docs = sorted(process_query(query, inverted_index, all_docs))[:5]
print(f"\nQuery: {query}\nMatching Documents:")
for doc_id in matching_docs:
    email_content = next(email["content"] for email in email_data if email["Document-ID"] == doc_id)
    print(f"{doc_id}: {' '.join(email_content)}")  # Print only first 30 words for preview


Query: winter OR summer
Matching Documents:
10353423.1075857652669: maybe hydro situation dire in west think water levels are at recent historical lows problem is from gas standpoint west is an island right now every molecle that can go there is so will provide limited support to prices in east hydro in east is actually very healthy would assume your markets are targeting eastern u s so i dont know if hydro problem in west is that relevant sarah mulholland am to john arnold hou ect ect cc subject re us fuel interesting comment from singapore hope things are going well up there forwarded by sarah mulholland hou ect on am hans wong am to sarah mulholland hou ect ect cc niamh clarke lon ect ect stewart peter lon ect ect caroline cronin eu enron enron angela saenz enron enronxgate subject re us fuel i was reading something interesting last week somewhere on states coping with the coming summer the report was on the amount of ice not huge enough from this winter to provide enough water for

Now show the results for the query: `buyers AND risk AND NOT crazy`

In [13]:
# your code here
query = "buyers AND risk AND NOT crazy"
matching_docs = sorted(process_query(query, inverted_index, all_docs))[:5]
print(f"\nQuery: {query}\nMatching Documents:")
for doc_id in matching_docs:
    email_content = next(email["content"] for email in email_data if email["Document-ID"] == doc_id)
    print(f"{doc_id}: {' '.join(email_content)}")  # Print only first 30 words for preview


Query: buyers AND risk AND NOT crazy
Matching Documents:
2726985.1075857655016: forwarded by john arnold hou ect on am from ann m schmidt enron on am to ann m schmidt corp enron enron cc bcc john arnold hou ect subject enron mentions utility deregulation square peg round hole the new york times executives considered to head military los angeles times bush leaning toward execs for military the seattle times enron s chief denies role as energy villain critics regard kenneth lay as deregulation opportunist the san francisco chronicle enron boss says he s not to blame for profits in energy crisis associated press newswires the stadium curse some stocks swoon after arena deals the san francisco chronicle money and business financial desk section economic view utility deregulation square peg round hole by joseph kahn the new york times page column c new york times company washington in the forensic pursuit of what caused california s power failure the bush administration the energy industry

Now show the results for the query: `never OR know`

In [14]:
# your code here
query = "never OR know"
matching_docs = sorted(process_query(query, inverted_index, all_docs))[:5]
print(f"\nQuery: {query}\nMatching Documents:")
for doc_id in matching_docs:
    email_content = next(email["content"] for email in email_data if email["Document-ID"] == doc_id)
    print(f"{doc_id}: {' '.join(email_content)}")  # Print only first 30 words for preview


Query: never OR know
Matching Documents:
10008095.1075857595829: not me brian hoskins enron communications pm to john arnold hou ect ect cc subject re never mind did you do that brian t hoskins enron broadband services office mobile fax brian_hoskins enron net forwarded by brian hoskins enron communications on pm john cheng enron sent by john cheng enron pm to john cheng na enron enron cc brian hoskins enron communications enron communications john cheng na enron enron enron communications fangming zhu corp enron enron enron communications pablo torres corp enron enron enron communications don adam corp enron enron enron communications marlin gubser hou ect ect mark hall hou ect subject re all sorry for the lengthy thread a service request has been opened with microsoft on the ie5 issues and her application regards jkc john cheng pm sent by john cheng to brian hoskins enron communications enron communications cc john cheng na enron enron enron communications fangming zhu corp enron en

## Observations (10 points)
Does your boolean search engine find relevant documents for these queries? As in, would our customers be happy if we shipped this retrieval engine? Why or why not?

What is the impact of the pre-processing options? Do they impact your search quality?

*your discussion here*

Yes, my boolean search engine does find relevant documents for these queries. But the main limitation of this boolean search engine is if a user’s query does not match exactly with indexed terms, the engine may return no results, even if relevant documents exist with slight variations in wording. Additionally, this boolean search engine treats all matching documents equally without prioritizing more relevant document.

Customer's happiness is based on the type of customers we will be dealing with. If our customers require precise control over document retrieval; for example legal, medical, compliance industries; they might find the boolean engine useful. However, for general users, a boolean search engine may feel rigid, leading to frustration due to zero-result queries or overwhelming results without relevance ranking.

Yes, pre-processing options does impact our search quality significantly. Without proper pre-processing, boolean search is too rigid. For example, in our case we haven't performed stemming and lemmatization which causes boolean search irrelevant as it will retrieve only the documents based on exact query which matches with our token.

# Part 3: Ranking (35 points)

For the third part, you will add ranking to your search system. Given a search query, our goal is to retrieve the top-5 most relevant emails by assigning a score to each document.

We will explore two ranking methods, each offering a different approach to scoring and ranking documents:


### Ranking method A: Ranking with vector space model with TF-IDF (15 points)

**Cosine:** You should use cosine as your scoring function.

**TFIDF:** For the **document vectors**, use the standard TF-IDF scores introduced in class. For the **query vector**, use **simple weights (the raw term frequency)**. For example:
* query: never $\rightarrow$ (1)
* query: never know $\rightarrow$ (1, 1)

**Query:**  `trade`

**Output:**
You should output the top-5 results plus the cosine score of each of these documents.  

The output should be like this:

Rank Scores DocumentID Document

---

You can additionally assume that your queries will contain at most three words. Be sure to normalize your vectors as part of the cosine calculation!


In [15]:
# your code here
import math
from collections import defaultdict

# Step 1: Compute TF
def compute_tf(doc_tokens):
    # Compute term frequency (TF) for a document.
    tf = defaultdict(float)
    total_terms = len(doc_tokens)
    for term in doc_tokens:
        tf[term] += 1
    for term in tf:
        tf[term] = math.log(1+tf[term], 2) # logarithmic base is 2
    return tf

# Step 2: Compute IDF
def compute_idf(documents):
    # Compute inverse document frequency (IDF) for the entire corpus.
    num_docs = len(documents)
    idf = defaultdict(float)
    doc_freq = defaultdict(int)

    for doc in documents:
        unique_terms = set(doc["content"])
        for term in unique_terms:
            doc_freq[term] += 1

    for term, freq in doc_freq.items():
        idf[term] = math.log(num_docs / (freq), 2) # logarithmic base is 2

    return idf

# Step 3: Compute TF-IDF for each document
def compute_tf_idf(documents, idf):

    # Compute TF-IDF vectors for all documents.
    tf_idf_vectors = {}
    for doc in documents:
        doc_id = doc["Document-ID"]
        tf = compute_tf(doc["content"])
        tf_idf_vectors[doc_id] = {term: tf[term] * idf[term] for term in tf}

    return tf_idf_vectors

# Step 4: Compute Cosine Similarity
def cosine_similarity(doc_vector, query_vector):
    # Compute cosine similarity between a document vector and a query vector.
    dot_product = sum(doc_vector.get(term, 0) * query_vector.get(term, 0) for term in query_vector)
    doc_magnitude = math.sqrt(sum(value**2 for value in doc_vector.values()))
    query_magnitude = math.sqrt(sum(value**2 for value in query_vector.values()))

    if doc_magnitude == 0 or query_magnitude == 0:
        return 0  # Avoid division by zero

    return dot_product / (doc_magnitude * query_magnitude)

# Step 5: Process Query and Rank Documents
def rank_documents(query, tf_idf_vectors, idf, documents):
    # Rank documents based on cosine similarity with the query.
    query_tokens = preprocess_email_content(query)  # Tokenize and preprocess query
    query_tf = compute_tf(query_tokens)  # Compute TF for query
    query_vector = {term: query_tf[term] * idf.get(term, 0) for term in query_tf}  # Compute query TF-IDF

    # Compute cosine similarity for each document
    scores = []
    for doc in documents:
        doc_id = doc["Document-ID"]
        doc_vector = tf_idf_vectors[doc_id]
        similarity = cosine_similarity(doc_vector, query_vector)
        scores.append((similarity, doc_id, doc["content"]))

    # Sort by descending similarity and return top-5 results
    top_5_results = sorted(scores, key=lambda x: x[0], reverse=True)[:5]

    print("\nTop-5 Retrieved Emails:")
    print("Rank   Score       Document-ID   Document \n" + "-" * 60)
    for rank, (score, doc_id, content) in enumerate(top_5_results, start=1):
        print(f"{rank:<6} {score:.4f}     {doc_id:<12} {' '.join(content[:20])}...")  # Print first 20 words

# Step 6: Run the Retrieval System
idf_scores = compute_idf(email_data)
tf_idf_vectors = compute_tf_idf(email_data, idf_scores)

# Example query
query = "trade"
rank_documents(query, tf_idf_vectors, idf_scores, email_data)


# print out the top-5 retrieved emails


Top-5 Retrieved Emails:
Rank   Score       Document-ID   Document 
------------------------------------------------------------
1      0.3053     15827855.1075857658654 torrey please set me up to trade crude john...
2      0.2994     32835197.1075857597302 hey i just want to confirm the trades i have in your book trade i sell x trade i buy...
3      0.2284     2752057.1075857658632 torrey can you also approve mike maggi to trade crude as well thanks for your help john...
4      0.1855     5340834.1075857658345 greg what is the correct formula you devised for profitability on last trade is mid...
5      0.1788     3383202.1075857656796 you fucker that s my trade i was trying to buy nines the last minutes all i got was scraps...


### Ranking method B: Ranking with BM25 (15 points)
Finally, let's try the BM25 approach for ranking. Refer to https://en.wikipedia.org/wiki/Okapi_BM25 for the specific formula. You could choose k_1 = 1.2 and b = 0.75 but feel free to try other options.

**Query:**  `gas floor`

**Output:**
You should output the top-5 results plus the BM25 score of each of these documents.  

The output should be like this:

Rank Scores DocumentID Document

In [16]:
# your code here

# BM25 Parameters
k1 = 1.2
b = 0.75

# Step 1: Compute IDF for BM25
def compute_bm25_idf(documents):
    # Compute Inverse Document Frequency (IDF) for BM25.
    num_docs = len(documents)
    idf = defaultdict(float)
    doc_freq = defaultdict(int)

    for doc in documents:
        unique_terms = set(doc["content"])  # Unique terms in the document
        for term in unique_terms:
            doc_freq[term] += 1  # Count documents containing term

    for term, freq in doc_freq.items():
        idf[term] = math.log((num_docs - freq + 0.5) / (freq + 0.5) + 1)  # BM25 IDF formula # Here the log base is e by default

    return idf

# Step 2: Compute BM25 Score for a Document
def compute_bm25_score(query_tokens, document, idf, avg_doc_len):
    # Compute BM25 score for a document given a query.
    doc_length = len(document["content"])
    tf = defaultdict(int)

    for term in document["content"]:
        tf[term] += 1  # Count term frequency in the document

    score = 0
    for term in query_tokens:
        if term in tf:
            term_freq = tf[term]
            numerator = term_freq * (k1 + 1)
            denominator = term_freq + k1 * (1 - b + b * (doc_length / avg_doc_len))
            score += idf[term] * (numerator / denominator)  # Apply BM25 formula

    return score

# Step 3: Rank Documents using BM25
def rank_documents_bm25(query, documents):
    # Rank documents using BM25 given a query.
    query_tokens = preprocess_email_content(query)  # Tokenize and preprocess query
    idf = compute_bm25_idf(documents)  # Compute IDF for BM25
    avg_doc_len = sum(len(doc["content"]) for doc in documents) / len(documents)  # Compute average doc length

    # Compute BM25 score for each document
    scores = []
    for doc in documents:
        doc_id = doc["Document-ID"]
        score = compute_bm25_score(query_tokens, doc, idf, avg_doc_len)
        scores.append((score, doc_id, doc["content"]))

    # Sort by descending score and return top-5 results
    top_5_results = sorted(scores, key=lambda x: x[0], reverse=True)[:5]

    return top_5_results

# Step 4: Run BM25 Ranking
query = "gas floor"
top_5_results = rank_documents_bm25(query, email_data)
print("\nTop-5 Retrieved Emails (BM25):")
print("Rank   Score       Document-ID   Document \n" + "-" * 60)
for rank, (score, doc_id, content) in enumerate(top_5_results, start=1):
    print(f"{rank:<6} {score:.4f}     {doc_id:<12} {' '.join(content[:20])}...")


# print out the top-5 retrieved emails


Top-5 Retrieved Emails (BM25):
Rank   Score       Document-ID   Document 
------------------------------------------------------------
1      7.6584     29559946.1075857598198 thx for the spreadsheet questions what time frame does this entail and does the correlation between the trader and agg...
2      6.8478     32732331.1075857597410 john i have asked mike and larry to spend half an hour each talking to you about opportunities on the...
3      6.8220     23846275.1075857658302 john i would like for you to come talk to a couple more people on the gas floor about a...
4      6.4052     8915800.1075857597866 thanks this is exactly what i wanted john enron north america corp from sunil dalal enron pm to john arnold...
5      6.0879     3417404.1075857651247 can you send jean a list of her seat numbers forwarded by john arnold hou ect on pm from jean...


Comparison between Cosine similarity function and BM25

In [17]:
# Retrieved top-5 emails for query="trade" for comparison
query = "trade"
top_5_results = rank_documents_bm25(query, email_data)
print("\nTop-5 Retrieved Emails (BM25):")
print("Rank   Score       Document-ID   Document \n" + "-" * 60)
for rank, (score, doc_id, content) in enumerate(top_5_results, start=1):
    print(f"{rank:<6} {score:.4f}     {doc_id:<12} {' '.join(content[:20])}...")


Top-5 Retrieved Emails (BM25):
Rank   Score       Document-ID   Document 
------------------------------------------------------------
1      4.5414     32835197.1075857597302 hey i just want to confirm the trades i have in your book trade i sell x trade i buy...
2      4.0640     3383202.1075857656796 you fucker that s my trade i was trying to buy nines the last minutes all i got was scraps...
3      3.9941     30793972.1075857600929 jim the list i gave you is a list of brokers that can clear through you not brokers that you...
4      3.9600     15827855.1075857658654 torrey please set me up to trade crude john...
5      3.9017     29790084.1075857652778 forwarded by john arnold hou ect on am zerilli frank fzerilli powermerchants com on am to jarnold enron com jarnold...


In [18]:
# Output for query = "trade" using Cosine similarity function:
# 1. 0.3053     15827855.1075857658654
# 2. 0.2994     32835197.1075857597302
# 3. 0.2284     2752057.1075857658632
# 4. 0.1855     5340834.1075857658345
# 5. 0.1788     3383202.1075857656796

In [19]:
# print length of terms in Doc-ID: 15827855.1075857658654
count = 0
for email in email_data:
    if email["Document-ID"] in ["15827855.1075857658654"]:
        print("Total number of terms in Doc-ID 15827855.1075857658654", len(email['content']))
        for text in email['content']:
          if text == 'trade':
            count += 1
print("Number of 'trade' term in Doc: ", count)

# print length of terms in Doc-ID: 32835197.1075857597302
count = 0
for email in email_data:
    if email["Document-ID"] in ["32835197.1075857597302"]:
        print("Total number of terms in Doc-ID 32835197.1075857597302", len(email['content']))
        for text in email['content']:
          if text == 'trade':
            count += 1
print("Number of 'trade' term in Doc: ", count)

count = 0
# print length of terms in Doc-ID: 2752057.1075857658632
for email in email_data:
    if email["Document-ID"] in ["2752057.1075857658632"]:
        print("Total number of terms in Doc-ID 2752057.1075857658632 ", len(email['content']))
        for text in email['content']:
          if text == 'trade':
            count += 1
print("Number of 'trade' term in Doc: ", count)

count = 0
# print length of terms in Doc-ID: 5340834.1075857658345
for email in email_data:
    if email["Document-ID"] in ["5340834.1075857658345"]:
        print("Total number of terms in Doc-ID 5340834.1075857658345 ", len(email['content']))
        for text in email['content']:
          if text == 'trade':
            count += 1
print("Number of 'trade' term in Doc: ", count)

count = 0
# print length of terms in Doc-ID: 3383202.1075857656796
for email in email_data:
    if email["Document-ID"] in ["3383202.1075857656796"]:
        print("Total number of terms in Doc-ID 3383202.1075857656796", len(email['content']))
        for text in email['content']:
          if text == 'trade':
            count += 1
print("Number of 'trade' term in Doc: ", count)

Total number of terms in Doc-ID 15827855.1075857658654 9
Number of 'trade' term in Doc:  1
Total number of terms in Doc-ID 32835197.1075857597302 41
Number of 'trade' term in Doc:  3
Total number of terms in Doc-ID 2752057.1075857658632  17
Number of 'trade' term in Doc:  1
Total number of terms in Doc-ID 5340834.1075857658345  15
Number of 'trade' term in Doc:  1
Total number of terms in Doc-ID 3383202.1075857656796 58
Number of 'trade' term in Doc:  2


## Observations (5 points)
What do you observe? Are there key differences between the two ranking approaches? Briefly discuss in bullet points.


* **Your observations:**

Here, we can see that the relevance ordering differs when we use Cosine and BM25 methods. To better understand the difference between them let's observe these 2 Doc-IDs:

1. Doc-ID 32835197.1075857597302 (**BM25 Rank 1, Cosine Rank 2**): Total terms: 41, 'trade' frequency: 3
2. Doc-ID 15827855.1075857658654 (**BM25 Rank 4, Cosine Rank 1**): Total terms: 9, 'trade' frequency: 1

We know that, BM25 prioritizes term frequency but accounts for document length normalization. For the first example(32835197.1075857597302), BM25 gives it the highest score even though the term “trade” appears only 3 times because it accounts for document length (41 words). This means BM25 likely down-weights term frequency in longer documents while still considering importance.

For the second document example(15827855.1075857658654), Cosine similarity ranks this document first because "trade" appears 1 time in just 9 words, giving it a high TF-IDF weight. However, BM25, ranks it lower (4th place) because it accounts for term saturation—just because a term appears 1 time in a very short document doesn’t mean it's always more relevant.

* **Differences:**

Through this observation we notice that BM25 and Cosine Similarity are both used to measure how relevant a document is to a search query, but they work in different ways.

Cosine Similarity focuses mainly on term frequency and gives higher scores to short documents where the search term appears frequently. However, this can sometimes lead to overly short documents being ranked too high, even if they are not the most relevant.

On the other hand, BM25 takes document length into account and prevents long documents from being unfairly penalized. If a term appears too often in a long document, BM25 reduces its impact to avoid giving too much importance to repetitive words. At the same time, it ensures that short documents do not dominate the rankings just because they contain the term frequently. Because of this, BM25 does a better job at mimicking how real-world search engines rank documents, making it more effective for retrieving useful search results.

---



# Part 4: Cool LLM RAG Extension (15 points)

Finally, we give you an opportunity to explore using OASIS for Retrieval-Augmented Generation (RAG) with LLMs.
Here, the task is to retrieve the top-5 emails for a query you like. You will then pass the retrieved email content along with your question to the LLM and let it answer your question. Specifically, we want you:

* Query the LLM directly with your question.
* Retrieve the top-5 emails based on your query and pass them to the LLM along with your question.
* How is the RAG output different from the non-RAG output? Does retrieval help the LLM better answer your question?

We recommend using Gemini following the [instructions](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Prompting.ipynb) here.

Hint: Take a close look at the dataset and pick a specific, relevant query where retrieval can enhance the LLM’s response.

*What You Will Submit*
- Your query.
- The top-5 retrieved emails (including Document ID and ranking score).
- The LLM's response without retrieval.
- The LLM's response with retrieval (RAG).
- A brief analysis comparing both outputs.

We will grade this last part according to correctness, effort, and creativity.

## Step 1: Query the LLM Directly (Without Retrieval) (5 points)

In [20]:
!pip install -U -q "google-generativeai>=0.7.2" # Install the Python SDK

In [21]:
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [22]:
import google.generativeai as genai  # Google Gemini API

# Gemini API Setup
def query_gemini(query_text):
    """Query Gemini LLM directly (without retrieval)."""
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)

    model = genai.GenerativeModel("gemini-pro")
    response = model.generate_content(query_text)
    return response.text

#  Run LLM Query
query_text = "Please reply"

# Step 1: Query LLM directly (Without Retrieval)
llm_response_no_rag = query_gemini(query_text)

#  Print Results
print("\n Query:", query_text)
print("\n LLM Response Without Retrieval:\n")
print(llm_response_no_rag)



 Query: Please reply

 LLM Response Without Retrieval:

Sure, I can help you with that. Please let me know what you need assistance with, and I'll do my best to provide a helpful response.


## Step 2: Query the LLM with Retrieved Emails (RAG) (5 points)

In [23]:
#  Gemini API Setup
def query_gemini_with_rag(query_text, retrieved_emails):
    """Query Gemini LLM with Retrieval-Augmented Generation (RAG)."""
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)

    model = genai.GenerativeModel("gemini-pro")

    # Prepare retrieved email context
    email_context = "\n\n".join([f"Email {i+1} (Doc {doc_id}): {content}"
                                for i, (_, doc_id, content) in enumerate(retrieved_emails)])

    # Construct prompt with retrieved emails
    prompt = f"Based on the following emails:\n\n{email_context}\n\nQuery: {query_text}\nAnswer:"

    response = model.generate_content(prompt)
    return response.text

#  Step 2: Retrieve Emails & Query LLM

query_text = "Please reply"

top_5_bm25_results = rank_documents_bm25(query_text, email_data)


# Step 2.2: Query LLM with Retrieved Emails (RAG)
llm_response_rag = query_gemini_with_rag(query_text, top_5_bm25_results)

#  Print Results

print("\n Query:", query_text)
print("\n Top-5 Retrieved Emails (BM25 Ranking):")
print("Rank   Score       Document-ID   Document \n" + "-" * 60)
for rank, (score, doc_id, content) in enumerate(top_5_bm25_results, start=1):
    print(f"{rank:<6} {score:.4f}     {doc_id:<12} {' '.join(content[:20])}...")  # Print first 20 words for brevity

print("\n LLM Response With Retrieval (RAG):\n")
print(llm_response_rag)


 Query: Please reply

 Top-5 Retrieved Emails (BM25 Ranking):
Rank   Score       Document-ID   Document 
------------------------------------------------------------
1      7.6356     4517077.1075857600583 vlade i have not heard back from you what is the schedule for changing the var process please reply john...
2      7.1060     24971240.1075857597324 received z zyft02 yahoo com on am please respond to zyft02 yahoo com to john arnold enron com cc subject...
3      6.6585     32485058.1075857601648 tanya on friday i emailed a new vol curve to use for var testing i was under the impression that...
4      6.5422     18936962.1075857656536 can you send it through lavo he s suppose to pay for it thanks john liz m taylor am to...
5      6.2484     2564966.1075857650834 no soblander carrfut com on am to soblander carrfut com cc subject please respond carr is hosting an enymex presentation...

 LLM Response With Retrieval (RAG):

john


## Discussion (5 points)

In this section, reflect on the performance of different ranking methods and the impact of retrieval on LLM responses. Consider:

- How did retrieval affect the LLM’s response? Did it improve factual accuracy or relevance?
- Were there cases where retrieval hurt performance (e.g., irrelevant documents, redundancy)?
- Any ideas for improving the ranking or retrieval process?

Keep your discussion *concise* and *insightful*, focusing on key takeaways from your experiments. Please answer in bullet points.

*your discussion here*

* ***Impact of Retrieval on LLM’s Response:***
Retrieval helped the LLM generate more factually accurate and relevant responses by providing it with high-quality, context-specific documents.
For example without retrieval the LLM responded giving generic answer as 'Sure, I can help you ....' but after providing it retrieved emails it gave the most relevant word to the query as 'John'

* ***Cases Where Retrieval Hurt Performance:***

1. If irrelevant or redundant documents are retrieved, they can mislead the LLM, introducing noise or unnecessary information into the response. For example, in this case, as humans, we understand that when we prompt the LLM with "Please reply," we expect it to either ask us what to reply or provide relevant information if it knows the context. This is exactly what the LLM without retrieval did. However, when the LLM had access to our retrieved emails, it generated a response solely based on the provided information and answered 'John' which is the most relevant word to the query 'Please reply'.

2. Appart from LLM part, in cosine similarity score determination, short documents with high term frequency (favored by Cosine Similarity) sometimes ranked too high, leading to responses that lacked depth.

* ***Ideas for Improving Ranking & Retrieval:***

1. Removing duplicate or irrelevant documents before retrieval could improve response quality.

2. Instead of raw frequency weights as 1, we could assign high values to the important words. This will help in ranking and getting correct relevance order even if the term frequency of that word/term is less.

3. Add stemming and lemmatization while preparing tokens. This will ensure better matching for query and documents. Moreover, it will reduce dictionary/vocabulary size.

# Collaboration Declarations

*You should fill out your collaboration declarations here.*

1. Refered this stackoverflow link to write BM25 code:
https://stackoverflow.com/questions/61877065/implementation-of-okapi-bm25-in-python

2. Refered this wikipedia link to correctly implement and understand BM25 formula:
https://en.wikipedia.org/wiki/Okapi_BM25

3. Refered this link to develop cosine similarity function and tf idf function with slight variations to formula according to the lecture slides:
https://medium.com/towards-data-science/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089

4. Used chatgpt while remove more special characters, punctions and other things in addition to the ones given in code prompt.

5. Cross checked outputs with friends and discussion forum.